In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from datetime import datetime, timedelta
import pickle
import ipywidgets as widgets
import cartopy.crs as ccrs
import cartopy.feature as cfeature

plt.rcParams['legend.fontsize'] = 10
plt.rcParams.update({'figure.autolayout': True})

## GPS_Stations Class
The `GPS_Stations` class is a wrapper for a data dictionary, `{Name: time series data}`. The time series data is an Nx4 ndarray, where N is the number of dates, from 2011/4/1 to 2016/12/31. The 4 columns correspond to date (`datetime` type), east, north and up components (all `float` type).

In [ ]:
class GPS_Stations():
    def __init__(self):
        self.data = {}
    def add(self, station_name, date, x, y, z):
        if station_name in self.data:
            # if the station is already in the data dictionary, append a new row of data [date,x,y,z]
            self.data.update( {station_name : np.append( self.data[station_name], np.array([date,x,y,z]).reshape(1,4), axis=0 ) } )
        else: # if first time adding this station, create a key:data pair.
            self.data.update( {station_name : np.array([date,x,y,z]).reshape(1,4)} )

## Loading Data
If a pickled data object exists, load and use it as an instance of `GPS_Stations` class named `Korea_GPS_Stations`. If not, i.e., `DATA_EXISTS==False`, read the data files retrived from the repository associated with (Kim et al., Tectonics, 2018) to create and populate an instance of `GPS_Stations` named `Korea_GPS_Stations`. 

If a pickled `GPS_Stations` data object exists, the first cell will just load it rather than read the coordinate files again. In this case, `DATA_EXISTS` is set to `True` so that the second cell can be skipped.

If such a pickled object does not exist (`DATA_EXISTS == False`), the second cell below should be run. Then, the newly created object is pickled for later use.

In [ ]:
DATA_EXISTS = False
try: 
    Korea_GPS_Stations = pickle.load( open( "Korea_GPS_Stations.p", "rb" ) )
    if Korea_GPS_Stations:
        DATA_EXISTS = True
        print("Data object exists")
except FileNotFoundError:
    print("Failed to load the pickled data. Move to the next cell.")

In [ ]:
if DATA_EXISTS == False:
    data_dir = "GPS Daily Coordinate data (2011 to 2016)/Coordinate"
    Korea_GPS_Stations = GPS_Stations()
    years = np.arange(2011,2017)
    #print(years)
    networks = np.array(['KASI', 'NGII'])
    #print(networks)
    for yr in years:
        for nt in networks:
            path = "./{0:s}/{1}/{2}".format(data_dir, yr, nt)
            #print(path)
            for f in os.listdir(path):
                fname = "{0}/{1}".format(path,f)
                fi = open(fname, 'r')
                lines = fi.readlines()
                date = np.datetime64(lines[2].split()[5]).astype(datetime)
                if date == datetime(2012,6,1): # exclude outlier
                    continue
                linecount = 7
                for line in lines[7:]:
                    linecount += 1
                    fields = line.split()
                    fieldnum = len(fields)
                    if fieldnum == 6:   # valid data without 'Name' field
                        name = fields[1]
                        x = float(fields[2])
                        y = float(fields[3])
                        z = float(fields[4])
                    elif fieldnum == 7: # valid data with all the fields filled
                        name = fields[1]
                        x = float(fields[3])
                        y = float(fields[4])
                        z = float(fields[5])
                    elif fieldnum == 5:   # invalid data with 0 0 0 coordinates
                        continue  # move on to the next line
                    elif fieldnum <= 1: # bottom blank lines
                        break     # stop processing this file
                    else:               # None of the above. Abort
                        print("Problem with line {0} in {1}".format(linecount, fname))
                        break     # stop processing this file
                    #print(fname, line)
                    Korea_GPS_Stations.add(name, date, x, y, z)
                fi.close()

    # Store the data object for quicker loading later
    with open("Korea_GPS_Stations.p", "wb") as handle:
        pickle.dump(Korea_GPS_Stations, handle, protocol=pickle.HIGHEST_PROTOCOL)
    DATA_EXISTS = True

In [ ]:
print( Korea_GPS_Stations.data['PUSN'][-1,1] - Korea_GPS_Stations.data['PUSN'][0,1] )
print( Korea_GPS_Stations.data['PUSN'][-1,0],Korea_GPS_Stations.data['PUSN'][0,0] )

In [ ]:
for station in Korea_GPS_Stations.data.keys():
    cmd = "wget "
    print(station)

## Data cleaning

### Jump removal
A function `remove_jump` is used for removing a jump in a time series.

In [ ]:
def remove_jump( stations, s, jump_date1, jump_date2 ):
    d = stations.data[s][:,0]
    e = stations.data[s][:,1]
    n = stations.data[s][:,2]
    u = stations.data[s][:,3]

    jump_ind1 = np.where(d == jump_date1)[0]
    jump_ind2 = np.where(d == jump_date2)[0]
    after_jump = d > jump_date1
    
    #print(jump_ind1, jump_ind2)
    #print(jump_ind2[0], jump_ind2[0]+5, stations.data[s][jump_ind2[0]:(jump_ind2[0]+5),1:4])
    avg_period = 20
    if s == 'JEOJ':
        avg_period = 120
    jumpval = np.sum(stations.data[s][jump_ind2[0]:(jump_ind2[0]+avg_period),1:4],\
                     axis=0)/avg_period
    print(jumpval)
    jump = jumpval - stations.data[s][jump_ind1,1:4]
    #jump = stations.data[s][jump_ind2,1:4]-stations.data[s][jump_ind1,1:4]
    #print(jump.shape)
    new_e = np.where(after_jump, e-jump[0,0], e)
    new_n = np.where(after_jump, n-jump[0,1], n)
    new_u = np.where(after_jump, u-jump[0,2], u)
    #plt.plot(d, new_u, color='red')
    #plt.plot(d, u)

    stations.data[s][:,1] = new_e
    stations.data[s][:,2] = new_n
    stations.data[s][:,3] = new_u

### Remove stations with incomplete time period

Most of the stations have a position time series spanning 2011/04/01 to 2016/12/31.
Any station with a shorter data span is removed from the `Korea_GPS_Stations object`'s data dictionary.

In [ ]:
# Verify the loaded data if desired.
stations_to_discard = []
for s in Korea_GPS_Stations.data.keys():
    if Korea_GPS_Stations.data[s][0,0] > datetime(2011,4,1).date() or \
        Korea_GPS_Stations.data[s][-1,0] < datetime(2016,12,31).date():
        stations_to_discard.append(s)
for s in stations_to_discard:
    Korea_GPS_Stations.data.pop(s)
    
scounter = 0
for s in Korea_GPS_Stations.data.keys():    
    scounter += 1
    print(scounter, s, Korea_GPS_Stations.data[s][0,0], Korea_GPS_Stations.data[s][-1,0])

### Removing outliers with Extended Linear Trajectory Models

#### `get_station_data()`
A function that returns date, easting, northing and up for a `station` from `start_datetime` to `end_datetime`.

#### `plot_selected_station()`
A function that plots easting, northing and up for a `selected_station` from `start_datetime` to `end_datetime`.

#### `eltm()` and `get_eltm()`
Implementation of Extended Linear Trajectory Model (Bevis and Brown, J.Geodesy, 2014):

$
\mathbf{x}(t) = \sum_{i=1}^{np+1} \mathbf{p}_{i}(t-t_{R})^{i-1} +
\sum_{j=1}^{nJ} \mathbf{b}_{j}H(t-t_{j}) + 
\sum_{k=1}^{nF} \left( \mathbf{s}_{k}\sin(\omega_{k}t) + \mathbf{c}_{k}\cos(\omega_{k}t) \right) +
\sum_{m=1}^{nT} \mathbf{a}_{m}\log(1+\Delta t_{m}/T_{m}).
$

Here, we consider only the following model with 11 parameters:
$
\mathbf{x}(t) = (pt+q) +
\left( \mathbf{s}_{a}\sin(\omega_{a}t) + \mathbf{c}_{a}\cos(\omega_{a}t) \right) +
\left( \mathbf{s}_{h}\sin(\omega_{h}t) + \mathbf{c}_{h}\cos(\omega_{h}t) \right) +
a\log(1+\Delta t_{m}/T_{m}).
$

The difference between `eltm` and `get_eltm` is that `eltm` takes `days` as `float` while `get_eltm` internally converts `datetime` to `float`.

#### `eltm_linear_only` and `get_eltm_linear_only()`
Only the linear component is assumed to contribute to tectonic displacements.

In [ ]:
def get_station_data( station, start_datetime=datetime(2011, 4, 1), end_datetime=datetime(2016, 12, 31) ):
    """
    Retrieves a station's data from start_datetime to end_datetime (all inclusive)
    and returns 4 arrays of daily time series, date, easting, northing and up.
    Note that position components are shifted to be zero on the starting date.
    """
    data = Korea_GPS_Stations.data[station]
    start_date = start_datetime.date() 
    if start_date < data[0,0]:
        start_date = data[0,0]
        print("Warning: Start date reset to {0}".format(start_date))
    end_date = end_datetime.date()
    if end_date > data[-1,0]:
        end_date = data[-1,0]
        print("Warning: End date reset to {0}".format(end_date))
    print(start_date, data[0,0], end_date, data[-1,0])
    
    datemask = np.logical_and(data[:,0] >= start_date, data[:,0] <= end_date)
    start_index = np.where(data[:,0] == start_date)[0]
    print("start_index = {}".format(start_index))
    if len(start_index) != 1:
        print("start_index = ", start_index)
        print("first and last date in the data are ", data[0,0], data[-1,0])
    
    date = data[datemask,0]
    easting  = data[start_index,1] - data[datemask,   1]
    northing = data[datemask,   2] - data[start_index,2]
    up       = data[datemask,   3] - data[start_index,3]
    print(data[start_index,2], data[start_index,3])
    print(len(date), len(data[:,0]))
    
    return date, easting, northing, up

def plot_selected_station( selected_station, start_datetime=datetime(2011, 9, 1), 
                          end_datetime=datetime(2016, 9, 30) ):
    """
    plot the east, north and up position time series for a selected station 
    from a start date until an end date.
    """
    d, e, n, u = get_station_data(selected_station, start_datetime, end_datetime)

    fig = plt.figure()
    ax = fig.add_subplot()

    ax.plot(d, e, 'ro', markersize=2, label="east")
    ax.plot(d, n, 'go', markersize=2, label="north")
    ax.plot(d, u, 'bo', markersize=2, label="up")
    ax.set_title(selected_station)
    ax.set_ylabel('Displacement (m)')
    ax.set_ylim((-0.1,0.1))
    ax.legend(loc=2)

In [ ]:
def eltm(days, p, q, sa, sp, ca, cp, sh, shp, ch, chp, a):
    from numpy import sin, cos, log   
    wa = 2.0*np.pi/365.0  # annual frequency
    wh = 4.0*np.pi/365.0  # semi-annual frequency
    # Number of days since Tohoku Eq (March 11, 2011) until April 1, 2011.
    dd_eq = 21.0 # = (datetime(2011, 4, 1)-datetime(2011, 3, 11)) / timedelta(days=1)
    dd = 2.0*(days + dd_eq)/365.0 # time scale: half a year
    disp = (p*days+q) + sa * sin(wa*(days-sp)) + ca * cos(wa*(days-cp)) \
            + sh * sin(wh*(days-shp)) + ch * cos(wh*(days-chp)) + a * log(1.0+dd)
    return disp

def get_eltm(day, coeff):
    days = day.astype(float)
    p, q, sa, sp, ca, cp, sh, shp, ch, chp, a = coeff
    from numpy import sin, cos, log   
    wa = 2.0*np.pi/365.0  # annual frequency
    wh = 4.0*np.pi/365.0  # semi-annual frequency
    # Number of days since Tohoku Eq (March 11, 2011) until April 1, 2011.
    dd_eq = 21.0 # = (datetime(2011, 4, 1)-datetime(2011, 3, 11)) / timedelta(days=1)
    dd = 2.0*(days + dd_eq)/365.0 # time scale: half a year
    disp = (p*days+q) + sa * sin(wa*(days-sp)) + ca * cos(wa*(days-cp)) \
            + sh * sin(wh*(days-shp)) + ch * cos(wh*(days-chp)) + a * log(1.0+dd)
    return disp

In [ ]:
def eltm_linear_only(days, p, q):
    disp = p*days+q
    return disp

def get_eltm_linear_only(day, coeff):
    days = day.astype(float)
    p, q = coeff
    disp = p*days+q
    return disp

In [ ]:
def plot_raw_e_eltm( selected_station, d, e, n, u, e_eltm, cleaned_station_data, popt ):
    fig, axs = plt.subplots(1,2)
    for ax in axs:
        for axis in ['top', 'bottom', 'left', 'right']:
            ax.spines[axis].set_linewidth(2.0)
    fig.set_figheight(4)
    fig.set_figwidth(9)

    axs[0].plot(d,e)
    axs[0].tick_params(direction='out', length=6, width=2, labelsize=12)
    axs[0].tick_params(which='minor', direction='out', length=4, width=2)
    axs[0].set_title("{0}: Raw data".format(selected_station), size=16)
    axs[0].set_xlabel('Time', size=14)
    axs[0].set_ylabel('Easting (m)', size=14)
    axs[0].grid()

    # Plot both data and the fitting function
    from numpy import log
    axs[1].plot(cleaned_station_data['days'], cleaned_station_data['e'],\
                marker='.', markeredgecolor='gray', markerfacecolor='gray', linestyle='none', alpha=1.0, label='Cleaned data')
    axs[1].set_ylim((-0.05,0.2))
    axs[1].plot(cleaned_station_data['days'], e_eltm, color='tab:red',\
            linewidth=2, label='ELTM')
    axs[1].plot(cleaned_station_data['days'], popt[0]*cleaned_station_data['dfloat'] + popt[1], \
             color='tab:orange', linewidth=2, label='linear')
    axs[1].plot(cleaned_station_data['days'], popt[0]*cleaned_station_data['dfloat'] + popt[1] \
             + popt[10]*log(1.0+2.0*(cleaned_station_data['dfloat'] + 20.0)/365.0),\
            color='tab:green', linewidth=2, label='linear+log')
    axs[1].tick_params(direction='out', length=6, width=2, labelsize=12)
    axs[1].tick_params(which='minor', direction='out', length=4, width=2)
    axs[1].set_title("{0}: Cleaned data and ELTM".format(selected_station), size=16)
    axs[1].set_xlabel('Time', size=14)
    axs[1].set_ylabel('Easting (m)', size=14)
    legend = axs[1].legend(loc=4, fontsize=12)
    legend.get_frame().set_linewidth(2)
    axs[1].grid()

    plt.savefig("./ELTM_plots/Cleaned_ELTM_{0}.png".format(selected_station))
    plt.close(fig)

In [ ]:
def plot_raw_e_eltm_linear_only( selected_station, d, e, n, u, e_eltm, \
                                cleaned_station_data, popt, cov, stage):
    fig, axs = plt.subplots(1,2)
    for ax in axs:
        for axis in ['top', 'bottom', 'left', 'right']:
            ax.spines[axis].set_linewidth(2.0)
    fig.set_figheight(4)
    fig.set_figwidth(9)

    axs[0].plot(d,e)
    axs[0].tick_params(direction='out', length=6, width=2, labelsize=12)
    axs[0].tick_params(which='minor', direction='out', length=4, width=2)
    if stage == 'A':
        axs[0].set_xticks([datetime(2011,4,1),datetime(2012,4,1),datetime(2013,4,1)])
    else:
        axs[0].set_xticks([datetime(2013,4,1),datetime(2015,4,1),datetime(2016,12,31)])
    axs[0].set_title("{0}: Raw data for Stage {1}".format(selected_station, stage), size=16)
    axs[0].set_xlabel('Time', size=12)
    axs[0].set_ylabel('Easting (m)', size=14)
    axs[0].grid()

    # Plot both data and the fitting function
    from numpy import log
    axs[1].plot(cleaned_station_data['days'], cleaned_station_data['e'],\
                marker='.', markeredgecolor='gray', markerfacecolor='gray', linestyle='none', alpha=1.0, label='Cleaned data')
    axs[1].set_ylim((-0.05,0.2))
#     axs[1].plot(cleaned_station_data['days'], e_eltm, color='tab:red',\
#             linewidth=2, label='ELTM')
    axs[1].plot(cleaned_station_data['days'], popt[0]*cleaned_station_data['dfloat'] + popt[1], \
             color='tab:orange', linewidth=2, label='linear fit')
    axs[1].tick_params(direction='out', length=6, width=2, labelsize=12)
    axs[1].tick_params(which='minor', direction='out', length=4, width=2)
    if stage == 'A':
        axs[1].set_xticks([datetime(2011,4,1),datetime(2012,4,1),datetime(2013,4,1)])
    else:
        axs[1].set_xticks([datetime(2013,4,1),datetime(2015,4,1),datetime(2016,12,31)])
    axs[1].set_title("{0}: Cleaned data for Stage {1}".format(selected_station, stage), size=16)
    axs[1].set_xlabel('Time', size=14)
    axs[1].set_ylabel('Easting (m)', size=14)
    legend = axs[1].legend(loc=4, fontsize=12)
    legend.get_frame().set_linewidth(2)
    axs[1].grid()
    
    vel = popt[0]*365.0e3 # m/day to mm/yr
    std = np.sqrt(np.diag(cov))[0]*365.0e3 # standard deviation in mm/yr
    vel_text = "Ve={0:.2f}".format(vel)
    std_text = "{0:.2f}".format(std)
    if stage == 'A':
        axs[1].text( datetime(2011,4,1), 0.175, vel_text+r'$\pm$'+std_text+' mm/yr', size=16, color='red')
    else:
        axs[1].text( datetime(2013,4,1), 0.175, vel_text+r'$\pm$'+std_text+' mm/yr', size=16, color='blue')

    plt.savefig("./ELTM_plots/As_Kim_etal_{0}{1}.png".format(selected_station, stage))
    plt.show()
    plt.close(fig)

### Main function that computes ELTM coefficients

In [ ]:
def get_eltm_as_Kim_etal_2018( selected_station, start_datetime, end_datetime ):
    """
    1. Retrieve days, east, north and up time series at a selected station 
    using 'get_station_data()'; 2. remove outlier relative to the full ELTM
    fitting; and 3. calculate coefficients for linear fitting to the cleaned
    data.
    Input: 
        - selected_station: A dictionary pairng selected station names
                            with raw position time series
        - start_datetime, end_datetime: start and end date of 'datetime' type
    Output:
        - ecoeff, ecov: ELTM coefficients and the associated covariance for
                        east components
        - ncoeff, ncov: for north components    
    """    
    d, e, n, u = get_station_data(selected_station, start_datetime, end_datetime)

    # Full ELTM fitting for outlier removal
    from scipy.optimize import curve_fit
    days_float = (d-start_datetime.date()) / timedelta(days=1)
    ecoeff, ecov = curve_fit(eltm, days_float, e)
    e_eltm = get_eltm(days_float, ecoeff)
    e_detrended = e - e_eltm
    ncoeff, ncov = curve_fit(eltm, days_float, n)
    n_eltm = get_eltm(days_float, ncoeff)
    n_detrended = n - n_eltm

    # Outlier detect/removal
    std_e = np.std(e_detrended)
    max_deviations = 1.0
    not_outlier = np.abs(e_detrended) < (max_deviations * std_e)

    # Store cleaned station data for another ELTM fitting
    cleaned_station_data = {}
    cleaned_station_data['days'] = d[not_outlier]
    cleaned_station_data['e'] = e[not_outlier]
    cleaned_station_data['n'] = n[not_outlier]
    cleaned_station_data['u'] = u[not_outlier]
    cleaned_station_data['dfloat'] = days_float[not_outlier].astype('float64')

    # Acquire ELTM again with the cleaned data
    ecoeff, ecov = curve_fit(eltm_linear_only, cleaned_station_data['dfloat'], cleaned_station_data['e'])
    e_eltm = get_eltm_linear_only(cleaned_station_data['dfloat'], ecoeff)
    ncoeff, ncov = curve_fit(eltm_linear_only, cleaned_station_data['dfloat'], cleaned_station_data['n'])
    n_eltm = get_eltm_linear_only(cleaned_station_data['dfloat'], ncoeff)
    
    stage = 'B'
    if start_datetime < datetime(2013,1,1):
        stage = 'A'
    plot_raw_e_eltm_linear_only( selected_station, d, e, n, u, e_eltm, \
                                cleaned_station_data, ecoeff, ecov, stage)
    
    return ecoeff, ecov, ncoeff, ncov

In [ ]:
# Station selector widget
station_selector = widgets.Select(
    options=Korea_GPS_Stations.data.keys(),
    # NOTE (rclam): seems this has to be manually changed on my end when change GPS station
    value='DAEJ', 
    rows=10,
    description='Select a GPS station:',
    disabled=False
)
display(station_selector)

In [ ]:
# When processing stations one by one:
selected_station = station_selector.value
ecoeffA, ecovA, ncoeffA, ncovA = get_eltm_as_Kim_etal_2018( selected_station, datetime(2011,4,1), datetime(2013,3,11) )
ecoeffB, ecovB, ncoeffB, ncovB = get_eltm_as_Kim_etal_2018( selected_station, datetime(2013,3,12), datetime(2016,12,31) )
print(ecoeffA*365.0e3, ecoeffB*365.0e3)
print(np.sqrt(np.diag(ecovA))*365.0e3, np.sqrt(np.diag(ecovB))*365.0e3)

- Data > 1 $\sigma$ from the detrended filtered out.
- Notes on 22 problematic stations
    - 3 stations are retained after jump removed:
        - HOMI, MARA, ULLE
    - 19 stations discarded
        - WPWN: Station unidentified and location unknown
        - JEOJ: Jump removed unsatisfactorily.
        - SOUL: Data corrupted after early 2015?
        - TABK: Data corrupted after early 2012?
        - DOKD: Data corrupted?
        - SEJO: Data length shorter than others.
        - GOJE: Data length shorter than others.
        - KUSN: Data length shorter than others.
        - YODK: Data length shorter than others.
        - JIND: Data length shorter than others.
        - YOIN: Data length shorter than others.
        - SEJN: Data length shorter than others.
        - GANH: Data length shorter than others.
        - BONH: Data length shorter than others.
        - DONH: Data length shorter than others.
        - CHUL: Data length shorter than others.
        - DANJ: Data length shorter than others.
        - GOSG: Data length shorter than others.
        - HCHN: Data length shorter than others.


In [ ]:
print(len(Korea_GPS_Stations.data.keys()))
problematic_stations = np.array(['JEOJ','SOUL','TABK','DOKD','SEJO','GOJE',\
                                 'KUSN','YODK','JIND','YOIN','SEJN','GANH',\
                                 'BONH','DONH','CHUL','DANJ','GOSG','HCHN','WPWN'])
for station in problematic_stations:
    try:
        Korea_GPS_Stations.data.pop(station)
        print("{0} removed".format(station))
    except KeyError:
        print("{0} already removed".format(station))
        continue
print(len(Korea_GPS_Stations.data.keys()))

In [ ]:
# Remove jump in some stations
remove_jump( Korea_GPS_Stations, 'HOMI', datetime(2013,8,4).date(), datetime(2013,8,5).date())
#remove_jump( Korea_GPS_Stations, 'JEOJ', datetime(2013,8,1).date(), datetime(2013,8,3).date())
remove_jump( Korea_GPS_Stations, 'MARA', datetime(2012,9,19).date(), datetime(2012,9,20).date())
remove_jump( Korea_GPS_Stations, 'ULLE', datetime(2014,9,16).date(), datetime(2014,9,17).date())

In [ ]:
# When processing all the stations in a batch:
counter = 0
station_eltm_data_AB = {}
for selected_station in Korea_GPS_Stations.data.keys():
    ecoeffA, ecovA, ncoeffA, ncovA = get_eltm_as_Kim_etal_2018( selected_station, \
                                                    datetime(2011,4,1), datetime(2013,3,11) )
    ecoeffB, ecovB, ncoeffB, ncovB = get_eltm_as_Kim_etal_2018( selected_station, \
                                                    datetime(2013,3,11), datetime(2016,12,31) )
    counter = counter + 1
    station_eltm_data_AB[selected_station] = {'ecoeffA':ecoeffA, 'ecovA':ecovA, \
                                              'ncoeffA':ncoeffA, 'ncovA':ncovA, \
                                              'ecoeffB':ecoeffB, 'ecovB':ecovB, \
                                              'ncoeffB':ncoeffB, 'ncovB':ncovB}
print(counter)

The data set with the problematic stations removed is pickled as `Korea_GPS_Stations_20110401_20161231.p`.
The linear and logarithmic coefficients and associated covariances from the ELTM fitting are saved as `Korea_GPS_Stations_coeff_cov_20110401_20161231.p`. Subsequent processing and analysis can start with loading it.

In [ ]:
# Save stations that has data from 2011/04/11 to 2016/09/11.
# with open('Korea_GPS_Stations_20110401_20161231.p', 'wb') as handle:
#     pickle.dump(Korea_GPS_Stations, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Korea_GPS_Stations_coeff_cov_AB.p', 'wb') as handle:
    pickle.dump(station_eltm_data_AB, handle, protocol=pickle.HIGHEST_PROTOCOL)